<a href="https://colab.research.google.com/github/omar-ahmd/Hackathon-NLP/blob/main/Livrable_AirIntelligence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importations utiles


In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import nltk
from nltk import SnowballStemmer
import spacy
import json
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.linalg import svds
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from scipy.sparse import hstack
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import f1_score, accuracy_score
import pickle


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Importation et preprocessing des données

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd
gdd.download_file_from_google_drive(file_id='1SlMDnllld8RTM2W9sz9LC9hJ62Ig01g1',
dest_path='./trainingData.csv'),gdd.download_file_from_google_drive(file_id='1Ad7GKKjHU7EO8TtrrI1NzxUp_R8L_uxh',
dest_path='./testData.csv')

(None, None)

In [ ]:
df = pd.read_csv('trainingData.csv',sep=";")
df = df.iloc[:,:-3] 
df=df[df.LIENWIKI.isna()==False]
df.LIENWIKI = df.LIENWIKI.str.replace("/wikisup/base%20de%20connaissance%20support/",'')
df.LIENWIKI[df.LIENWIKI.str.contains("host")] = "host"
df.ENONCE_PROBLEME_P = df.ENONCE_PROBLEME_P.str.replace("description de",'')
df = df.apply(lambda x: x.astype(str).str.lower())
df.drop(["STATUT","INCIDENT","DESIGNATION_PRODUIT","DESIGNATION_EQUIPEMENT","DESIGNATION_APP","CODE_PRODUIT","CODE_EQUIPEMENT","CODE_APP"],axis=1,inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
cols = df.select_dtypes(include=[np.object]).columns
df[cols[:-1]] = df[cols[:-1]].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
df[cols[:-1]] = df[cols[:-1]].apply(lambda x: x.str.replace('[^\dA-Za-z]', ' '))
df[cols[:-1]] = df[cols[:-1]].apply(lambda x: x.str.replace('xx', ''))
df[cols[:-1]] = df[cols[:-1]].apply(lambda x: x.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' '))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """


In [ ]:
le = preprocessing.LabelEncoder()
le.fit(df.LIENWIKI.unique())
df.LIENWIKI = le.transform(df.LIENWIKI)

In [ ]:
## Removing stopwords
list2 = ['utilisateur','wiki','description','lien','utilise','ok','probleme','actions','depuis','quand','oui','non','si','autre','ca','aujourd','hui','auj','u','lutilisateur','personne','autres','utilisateurs','car']
from nltk.corpus import stopwords
stp_list = stopwords.words('english') + stopwords.words('french') 
df['ENONCE_PROBLEME_P'] = df['ENONCE_PROBLEME_P'].apply(lambda x: ' '.join([item for item in x.split() if item not in stp_list]))

#Extraction des features


In [ ]:
cv1 = TfidfVectorizer(lowercase=True, min_df=3,  max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3))
                      #stop_words= stop_wordss)

vectorized1 = cv1.fit_transform(df.loc[:,"DESIGNATION"])

In [ ]:
cv2 = TfidfVectorizer(lowercase=True, min_df=10,  max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3))
                      #stop_words= stop_wordss)

vectorized2 = cv2.fit_transform(df.loc[:,"ENONCE_PROBLEME_P"])

In [ ]:
cv3 = TfidfVectorizer(lowercase=True, min_df=3,  max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3))
                      #stop_words= stop_wordss)

vectorized3 = cv3.fit_transform(df.loc[:,"NOMENCLATURE_PROBLEME"])

In [ ]:
Train_features = hstack([vectorized1,vectorized2,vectorized3])

# Training

In [ ]:
# SVC + kernel 
svc = SVC(kernel='rbf', C=12.5, gamma=0.58, probability=True)
svc.fit(Train_features, df.LIENWIKI)

SVC(C=12.5, gamma=0.58, probability=True)

In [ ]:
filename = 'finalizedSVC_model.sav'
pickle.dump(svc, open(filename, 'wb'))

# Testing

In [ ]:
test_df = pd.read_csv('testData.csv',sep=";")
(test_df.iloc[:,:]).head()


,INCIDENT,STATUT,DESIGNATION,CODE_PRODUIT,DESIGNATION_PRODUIT,CODE_EQUIPEMENT,DESIGNATION_EQUIPEMENT,CODE_APP,DESIGNATION_APP,NOMENCLATURE_PROBLEME,ENONCE_PROBLEME_P,LIENWIKI
0,639492,Archivé,#ADMINISTRATION/COMPTE/REINITIALISATION MOT DE...,NaN,NaN,11267.0,OPTIPLEX 3020SFF,1344.0,ADMINTOOLS,"Acces, Comptes",description de l'incident:l'user signale que q...,/wikisup/base%20de%20connaissance%20support/co...
1,644449,Archivé,#MATERIEL/TELEPHONIE/CISCO WIFI,454308.0,LECTEUR MINI FLASH USB LK2,216315.0,NaN,0.0,Sans Objet,Telephonie,description de l'incident :le capot arriere du...,/wikisup/base%20de%20connaissance%20support/su...
2,645665,Archivé,#MATERIEL/TELEPHONIE/CISCO WIFI,NaN,NaN,63080.0,Désignation nulle,0.0,Sans Objet,Pb fonctionnalites telephoniques,description de l'incident :l'utilisateur nous ...,/wikisup/base%20de%20connaissance%20support/su...
3,651210,Archivé,#MATERIEL/TELEPHONIE/IP,NaN,NaN,63110.0,LENOVO THINKCENTRE M700,0.0,Sans Objet,ToIP,description de l'incident : l'utilisateur me s...,/wikisup/base%20de%20connaissance%20support/su...
4,655283,Archivé,#MATERIEL/TELEPHONIE/CISCO WIFI,334897.0,DOUCHETTE DATALOGIC TOUCH 90 PRO,190848.0,Désignation nulle,0.0,Sans Objet,ToIP,description de l'incident : l'utilisateur a me...,/wikisup/base%20de%20connaissance%20support/su...


In [ ]:
# Preprocessing

test_df = test_df[test_df.LIENWIKI.isna()==False]
test_df.LIENWIKI = test_df.LIENWIKI.str.replace("/wikisup/base%20de%20connaissance%20support/",'')
test_df.ENONCE_PROBLEME_P = test_df.ENONCE_PROBLEME_P.str.replace("description de",'')
test_df = test_df.apply(lambda x: x.astype(str).str.lower())
#df["text"] = df.DESIGNATION + " " + df.NOMENCLATURE_PROBLEME + " " + df.ENONCE_PROBLEME_P
test_df.drop(["STATUT","INCIDENT","DESIGNATION_PRODUIT","DESIGNATION_EQUIPEMENT","DESIGNATION_APP","CODE_PRODUIT","CODE_EQUIPEMENT","CODE_APP"],axis=1,inplace=True)

cols = test_df.select_dtypes(include=[np.object]).columns
test_df[cols[:-1]] = test_df[cols[:-1]].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
test_df[cols[:-1]] = test_df[cols[:-1]].apply(lambda x: x.str.replace('[^\dA-Za-z]', ' '))
test_df[cols[:-1]] = test_df[cols[:-1]].apply(lambda x: x.str.replace('xx', ''))
test_df[cols[:-1]] = test_df[cols[:-1]].apply(lambda x: x.str.replace(r'\b\w\b', '').str.replace(r'\s+', ' '))





/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version.
  if sys.path[0] == '':
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [ ]:
test_df.LIENWIKI[test_df.LIENWIKI.str.contains("host")] = "host"

In [ ]:
test_df.LIENWIKI = le.transform(test_df.LIENWIKI)

test_df['ENONCE_PROBLEME_P'] = test_df['ENONCE_PROBLEME_P'].apply(lambda x: ' '.join([item for item in x.split() if item not in stp_list]))

In [ ]:
# Extraction des features

test_vectorized1 = cv1.transform(test_df.loc[:,"DESIGNATION"])
test_vectorized2 = cv2.transform(test_df.loc[:,"ENONCE_PROBLEME_P"])
test_vectorized3 = cv3.transform(test_df.loc[:,"NOMENCLATURE_PROBLEME"])

Test_features = hstack([test_vectorized1,test_vectorized2,test_vectorized3])

In [ ]:
# Importation du modèle
filename = 'finalizedSVC_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
Test_features.toarray().shape

(13464, 15642)

In [ ]:
ypred_proba= loaded_model.predict_proba(Test_features)
y_test = test_df.LIENWIKI

In [ ]:

best_indexes = np.argpartition(ypred_proba,np.argmin(ypred_proba,axis=1))[:,-3:]
first = loaded_model.classes_[best_indexes[:,-1]]
second = loaded_model.classes_[best_indexes[:,-2]]
third = loaded_model.classes_[best_indexes[:,-3]]
Top_3 = [first, second, third]
ypred_t1 = (y_test == first)
t2 = (y_test == second) 
t3 = (y_test == third)
ypred=np.logical_or(ypred_t1,t2)
ypred_t3 = np.logical_or(ypred,t3)

In [ ]:
print("Top 1:")
print(f"Accuracy score: {accuracy_score(y_test,first)}")
print("f1 score: {}".format(f1_score(y_test,first,average="macro")))
print("Top 3:")
print(f"Accuracy score: {ypred_t3.sum()/ypred_t3.shape[0]}")
#print("f1 score: {}".format(f1_score(y_test,first,average="macro")))

Top 1:
Accuracy score: 0.9726678550207962
f1 score: 0.9677533762043978
Top 3:
Accuracy score: 0.9931669637551991
